In [14]:
import os

os.chdir("/notebooks/bert-vits2/Bert-VITS2-Cantonese")

from models import (
    SynthesizerTrn,
)
import math
import torch
from text.symbols import symbols, num_tones, num_languages, yue_symbols
import utils
from torch import nn

In [48]:
hps = utils.get_hparams_from_file("/notebooks/bert-vits2/dataset/commonvoice16_1_bertvits2/configs/config.json")
mas_noise_scale_initial = 0.0
noise_scale_delta = 0.0

def load_model(model_file: str):
    net_g = SynthesizerTrn(
        len(symbols),
        hps.data.filter_length // 2 + 1,
        hps.train.segment_size // hps.data.hop_length,
        n_speakers=hps.data.n_speakers,
        mas_noise_scale_initial=mas_noise_scale_initial,
        noise_scale_delta=noise_scale_delta,
        **hps.model,
    )

    optim_g = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, net_g.parameters()),
        hps.train.learning_rate,
        betas=hps.train.betas,
        eps=hps.train.eps,
    )

    _, optim_g, g_resume_lr, epoch_str = utils.load_checkpoint(
        model_file,
        net_g,
        optim_g,
        skip_optimizer=(
            hps.train.skip_optimizer if "skip_optimizer" in hps.train else True
        ),
    )

    return net_g, optim_g

In [49]:
load_model("/notebooks/bert-vits2/dataset/commonvoice16_1_bertvits2/backup_models/G_0.pth")

/opt/conda/envs/peft/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
ERROR:utils:enc_p.emb.weight is not in the checkpoint
ERROR:utils:enc_p.tone_emb.weight is not in the checkpoint
ERROR:utils:enc_p.language_emb.weight is not in the checkpoint
ERROR:utils:enc_p.yue_bert_proj.weight is not in the checkpoint
ERROR:utils:enc_p.yue_bert_proj.bias is not in the checkpoint
ERROR:utils:emb_g.weight is not in the checkpoint


(SynthesizerTrn(
   (enc_p): TextEncoder(
     (emb): Embedding(155, 192)
     (tone_emb): Embedding(19, 192)
     (language_emb): Embedding(4, 192)
     (bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (ja_bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (en_bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (yue_bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (encoder): Encoder(
       (spk_emb_linear): Linear(in_features=512, out_features=192, bias=True)
       (drop): Dropout(p=0.1, inplace=False)
       (attn_layers): ModuleList(
         (0-5): 6 x MultiHeadAttention(
           (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (drop): Dropout(p=0.1, inplace=False)
         )
       )
      

In [2]:
checkpoint_dict = torch.load("/notebooks/bert-vits2/dataset/commonvoice16_1_bertvits2/backup_models/G_0.pth", map_location="cpu")

def resize_embedding_layer(weight, new_size):
    old_vocab_size = weight.size(0)
    if new_size < old_vocab_size:
        return weight[:new_size, :]
    elif new_size == old_vocab_size:
        return weight
    else:
        new_weight = weight.new_zeros(new_size - old_vocab_size, weight.size(1))
        embedding_dim = weight.size(1)
        avg_weight = weight.mean(dim=0, keepdim=True)
        noise_weight = torch.empty_like(new_weight)
        noise_weight.normal_(mean=0, std=(1.0 / math.sqrt(embedding_dim)))
        new_weight = avg_weight + noise_weight

        return torch.cat([weight, new_weight], dim=0)

In [3]:
checkpoint_dict['model']['enc_p.emb.weight'] = resize_embedding_layer(checkpoint_dict['model']['enc_p.emb.weight'], len(symbols))
checkpoint_dict['model']['enc_p.tone_emb.weight'] = resize_embedding_layer(checkpoint_dict['model']['enc_p.tone_emb.weight'], num_tones)
checkpoint_dict['model']['enc_p.language_emb.weight'] = resize_embedding_layer(checkpoint_dict['model']['enc_p.language_emb.weight'], num_languages)

In [47]:
torch.save(checkpoint_dict, "/notebooks/bert-vits2/dataset/commonvoice16_1_bertvits2/resized_models/G_0.pth")

In [50]:
load_model("/notebooks/bert-vits2/dataset/commonvoice16_1_bertvits2/resized_models/G_0.pth")

ERROR:utils:enc_p.yue_bert_proj.weight is not in the checkpoint
ERROR:utils:enc_p.yue_bert_proj.bias is not in the checkpoint
ERROR:utils:emb_g.weight is not in the checkpoint


(SynthesizerTrn(
   (enc_p): TextEncoder(
     (emb): Embedding(155, 192)
     (tone_emb): Embedding(19, 192)
     (language_emb): Embedding(4, 192)
     (bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (ja_bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (en_bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (yue_bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
     (encoder): Encoder(
       (spk_emb_linear): Linear(in_features=512, out_features=192, bias=True)
       (drop): Dropout(p=0.1, inplace=False)
       (attn_layers): ModuleList(
         (0-5): 6 x MultiHeadAttention(
           (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
           (drop): Dropout(p=0.1, inplace=False)
         )
       )
      

In [17]:
# Check the symbols

from text import cleaned_text_to_sequence

# test symbols
lines = []
my_phones = set()

with open("/notebooks/bert-vits2/dataset/commonvoice16_1_bertvits2/esd.list.cleaned", encoding="utf-8") as f:
    lines.extend(f.readlines())

for line in lines:
  wav_path, _, language_str, text, phones, tone, word2ph = line.strip().split("|")
  phone = phones.split(" ")
  my_phones.update(phone)
  # tone = [int(i) for i in tone.split(" ")]
  # word2ph = [int(i) for i in word2ph.split(" ")]
  # word2ph = [i for i in word2ph]
  # phone, tone, language = cleaned_text_to_sequence(phone, tone, language_str)

my_phones - set(yue_symbols)

{'!', "'", ',', '-', '.', '?', '_', 'z', '…'}